# Universal Pre-processing Notebook

This notebook shows the pre-processing steps for both datasets: 
 - Russian Troll Dataset (Github)
 - Manually pulled Verified Tweets using the Twitter API Academic License

Pre-processing is done separately due to the nature of the datasets and wanting to leave additional features available for EDA.

This notebook assumes that the "Merge_JSON" notebook has already been run to generate a merged json file.

In [1]:
import pandas as pd 
import os
import numpy as np
import demoji
import json

## Troll Tweets CSV Pre-processing

In [2]:
#load csv for troll dataset
all_data=pd.read_csv("../data/all_tweets_encoded.csv", encoding='UTF-8',low_memory=False)

In [5]:
all_data.shape

(2946207, 22)

In [6]:
pd.set_option('display.max_columns', None)
all_data.head(1)

,Unnamed: 0,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,article_url,tco1_step1,tco2_step1,tco3_step1
0,0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,RightTroll,0,905874659358453760,914580356430536707,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914580356430...,NaN,NaN


In [7]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2946207 entries, 0 to 2946206
Data columns (total 22 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   Unnamed: 0          int64 
 1   external_author_id  object
 2   author              object
 3   content             object
 4   region              object
 5   language            object
 6   publish_date        object
 7   harvested_date      object
 8   following           int64 
 9   followers           int64 
 10  updates             int64 
 11  post_type           object
 12  account_type        object
 13  retweet             int64 
 14  account_category    object
 15  new_june_2018       int64 
 16  alt_external_id     object
 17  tweet_id            int64 
 18  article_url         object
 19  tco1_step1          object
 20  tco2_step1          object
 21  tco3_step1          object
dtypes: int64(7), object(15)
memory usage: 494.5+ MB


#### Step 1: Filter for English Language Tweets Only (for NLP model)

In [9]:
all_data.language.value_counts()

English                2116867
Russian                 610943
German                   86983
Ukrainian                38669
Italian                  18063
Serbian                   9480
Uzbek                     9334
Bulgarian                 9236
LANGUAGE UNDEFINED        8320
Arabic                    7588
Macedonian                5178
French                    4562
Spanish                   3011
Norwegian                 2237
Farsi (Persian)           1684
Romanian                  1629
Dutch                     1165
Swedish                   1014
Vietnamese                 982
Estonian                   770
Japanese                   764
Catalan                    589
Finnish                    579
Albanian                   572
Croatian                   544
Icelandic                  498
Pushto                     487
Portuguese                 463
Turkish                    445
Lithuanian                 416
Polish                     386
Somali                     276
Slovak  

In [8]:
English_tweets = all_data[all_data['language']=='English']

In [38]:
cols_to_keep = ['external_author_id','author','content','region','language','publish_date','following','followers','updates',
               'retweet','account_category','tweet_id','tco1_step1']

In [39]:
English_tweets = English_tweets[cols_to_keep]

In [41]:
len(English_tweets.columns)

13

In [46]:
#Rename columns to match JSON
English_tweets = English_tweets.rename(columns={"retweet":"is_retweet", "tco1_step1":"full_url"})

In [42]:
#write to file for ease of import later
English_tweets.to_csv("../data/English_tweets.csv")

## JSON Tweets Pre-processing

In [3]:
#load merged json dataset from Twitter API
with open('../Tweet_Downloads/tweet_master.json', encoding='UTF-8') as json_file:
    tweet_master = json.load(json_file)

In [18]:
#convert json to pandas dataframe using normalize to flatten dict
df= pd.json_normalize(tweet_master)

### Step 1: Identify Relevant Cols

In [19]:
df.shape

(493186, 57)

In [20]:
pd.set_option('display.max_columns', None)

In [21]:
df.head(2)

,author_id,id,lang,conversation_id,edit_history_tweet_ids,reply_settings,created_at,context_annotations,source,text,possibly_sensitive,media,edit_controls.edits_remaining,edit_controls.is_edit_eligible,edit_controls.editable_until,attachments.media_keys,entities.annotations,entities.mentions,entities.urls,entities.hashtags,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,author.description,author.public_metrics.followers_count,author.public_metrics.following_count,author.public_metrics.tweet_count,author.public_metrics.listed_count,author.id,author.username,author.verified,author.created_at,author.name,author.pinned_tweet_id,author.profile_image_url,author.protected,in_reply_to_user_id,referenced_tweets,geo.country,geo.geo.type,geo.geo.bbox,geo.id,geo.country_code,geo.name,geo.place_type,geo.full_name,author.entities.url.urls,author.entities.description.urls,author.url,author.location,author.entities.description.mentions,entities.cashtags,attachments.poll_ids,geo.coordinates.type,geo.coordinates.coordinates,author.entities.description.hashtags
0,22203756,930982584649175047,en,930982584649175047,[930982584649175047],everyone,2017-11-16T02:15:19.000Z,"[{'domain': {'id': '10', 'name': 'Person', 'de...",Twitter for iPhone,Great meeting w/ GOP Govs at @The_RGA on @real...,False,"[{'type': 'photo', 'media_key': '3_93098256771...",5,True,2017-11-16T02:45:19.000Z,[3_930982567712641025],"[{'start': 278, 'end': 284, 'probability': 0.3...","[{'start': 41, 'end': 57, 'username': 'realDon...","[{'start': 286, 'end': 309, 'url': 'https://t....","[{'start': 277, 'end': 285, 'tag': 'RGA2017'}]",544,2217,3783,102,"Husband, father, 48th Vice President of the Un...",5835052,59,15232,11683,22203756,Mike_Pence,True,2009-02-27T23:04:51.000Z,Mike Pence,1551931068873424899,https://pbs.twimg.com/profile_images/137229142...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22203756,930954564769124352,en,930954365334097921,[930954564769124352],everyone,2017-11-16T00:23:59.000Z,"[{'domain': {'id': '10', 'name': 'Person', 'de...",Twitter for iPhone,.@realdonaldtrump's vision for tax cuts will d...,False,NaN,5,True,2017-11-16T00:53:59.000Z,NaN,NaN,"[{'start': 1, 'end': 17, 'username': 'realDona...",NaN,"[{'start': 209, 'end': 217, 'tag': 'RGA2017'}]",369,925,2919,74,"Husband, father, 48th Vice President of the Un...",5835052,59,15232,11683,22203756,Mike_Pence,True,2009-02-27T23:04:51.000Z,Mike Pence,1551931068873424899,https://pbs.twimg.com/profile_images/137229142...,False,22203756,"[{'type': 'replied_to', 'id': '930954365334097...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df.columns

Index(['author_id', 'id', 'lang', 'conversation_id', 'edit_history_tweet_ids',
       'reply_settings', 'created_at', 'context_annotations', 'source', 'text',
       'possibly_sensitive', 'media', 'edit_controls.edits_remaining',
       'edit_controls.is_edit_eligible', 'edit_controls.editable_until',
       'attachments.media_keys', 'entities.annotations', 'entities.mentions',
       'entities.urls', 'entities.hashtags', 'public_metrics.retweet_count',
       'public_metrics.reply_count', 'public_metrics.like_count',
       'public_metrics.quote_count', 'author.description',
       'author.public_metrics.followers_count',
       'author.public_metrics.following_count',
       'author.public_metrics.tweet_count',
       'author.public_metrics.listed_count', 'author.id', 'author.username',
       'author.verified', 'author.created_at', 'author.name',
       'author.pinned_tweet_id', 'author.profile_image_url',
       'author.protected', 'in_reply_to_user_id', 'referenced_tweets',
      

In [23]:
len(df.columns)

57

#### Creating column lists: 1st for EDA and another for merging the datasets

In [65]:
cols_keep_EDA = ['author_id','created_at','id','text','lang','author.location','author.name',
             'public_metrics.retweet_count','public_metrics.reply_count','public_metrics.like_count',
             'public_metrics.quote_count','author.username','author.public_metrics.followers_count',
             'author.public_metrics.following_count','author.entities.url.urls','author.created_at',
             'author.verified','context_annotations','entities.annotations','entities.mentions',
             'entities.hashtags', 'entities.urls']

In [66]:
df1 = df[cols_keep_EDA]

In [67]:
len(df1.columns)

22

### Step 2: Extract Feature Cols to match Troll Dataset Columns

**Feature: Is_Retweet**

In [68]:
#add new column with binary 1/0 if tweet is Retweet
df1['is_retweet']= np.where(df1.text.str.startswith('RT @'),1,0)

<ipython-input-68-d690f7b02df7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['is_retweet']= np.where(df1.text.str.startswith('RT @'),1,0)


In [69]:
df1['is_retweet'].value_counts()

0    409630
1     83556
Name: is_retweet, dtype: int64

**Feature: Updates**

In [70]:
#cols to add together to match the main dataset column "updates"
update_cols = ['public_metrics.retweet_count',
       'public_metrics.reply_count', 'public_metrics.like_count',
       'public_metrics.quote_count']

In [71]:
df1['updates']= df1[update_cols].sum(axis=1)

<ipython-input-71-442679c96d10>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['updates']= df1[update_cols].sum(axis=1)


**Feature: account_category**

In [72]:
#add Account_type column to represent verified account
df1['account_category']= df1['author.verified'].apply(lambda x: 'Verified_User' if True else 0)

<ipython-input-72-e359a2820873>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['account_category']= df1['author.verified'].apply(lambda x: 'Verified_User' if True else 0)


In [73]:
df1['account_category'].value_counts()

Verified_User    493186
Name: account_category, dtype: int64

### Step 3: Column updates

In [86]:
## change column names to match original dataset for merging
df2=df1.rename(columns={"author_id": "external_author_id", "created_at": "publish_date", "text": "content",
                  "lang": "language", "author.location":"region", "author.username":"author",
                  "author.name":"full_name","author.public_metrics.followers_count": "followers",
                  "author.public_metrics.following_count": "following",
                  "id": "tweet_id","entities.urls":"full_url"}, errors="raise")

In [87]:
df2.columns

Index(['external_author_id', 'publish_date', 'tweet_id', 'content', 'language',
       'region', 'full_name', 'public_metrics.retweet_count',
       'public_metrics.reply_count', 'public_metrics.like_count',
       'public_metrics.quote_count', 'author', 'followers', 'following',
       'author.entities.url.urls', 'author.created_at', 'author.verified',
       'context_annotations', 'entities.annotations', 'entities.mentions',
       'entities.hashtags', 'full_url', 'is_retweet', 'updates',
       'account_category'],
      dtype='object')

In [88]:
#drop columns for merging
df3= df2.drop(columns=['public_metrics.retweet_count',
       'public_metrics.reply_count', 'public_metrics.like_count',
       'public_metrics.quote_count','author.entities.url.urls', 'author.created_at', 'author.verified',
       'context_annotations', 'entities.annotations', 'entities.mentions',
       'entities.hashtags','full_name'])

In [89]:
df3.columns

Index(['external_author_id', 'publish_date', 'tweet_id', 'content', 'language',
       'region', 'author', 'followers', 'following', 'full_url', 'is_retweet',
       'updates', 'account_category'],
      dtype='object')

In [90]:
len(df3.columns)

13

In [93]:
sorted(English_tweets.columns) == sorted(df3.columns)

True

### Step 4: Merging the two datasets

In [95]:
merged = pd.concat([English_tweets,df3], axis=0)
merged.head()

,external_author_id,author,content,region,language,publish_date,following,followers,updates,is_retweet,account_category,tweet_id,full_url
0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,1052,9636,253,0,RightTroll,914580356430536707,https://twitter.com/10_gop/status/914580356430...
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,1054,9637,254,0,RightTroll,914621840496189440,https://twitter.com/damienwoody/status/9145685...
2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,1054,9637,255,1,RightTroll,914623490375979008,https://twitter.com/10_gop/status/913231923715...
3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,1062,9642,256,0,RightTroll,914639143690555392,https://twitter.com/10_gop/status/914639143690...
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,1050,9645,246,1,RightTroll,914312219952861184,https://twitter.com/realDonaldTrump/status/914...


In [96]:
merged.shape

(2610053, 13)

## Merged DF Pre-Processing

### Step 1: Add new features

**Feature: has_URL**

In [98]:
#Extract hyperlink from text
#add new column with binary 1/0 if tweet is Retweet
merged['has_URL']= np.where(merged.content.str.contains('http'),1,0)

In [99]:
merged.has_URL.value_counts()

0    2273265
1     336788
Name: has_URL, dtype: int64

**Feature: Emoji_text**

In [101]:
''' The following converts a text string with emojis into a list of descriptive text strings.
    Duplicate emojis are captured as each emoji converts to 1 text string.'''
def convert_emoji_list(x):
    lst=[]
    estring = ''
    import demoji
    import numpy as np
    if x is not np.nan:
        #extract list of text from demoji func
        lst = demoji.findall_list(x)
        if len(lst)<0:
            return np.nan
     
        else:
            return(lst)

In [102]:
# TEST FUNCTION FIRST
test = merged[:5]
test['emoji_text']= test['content'].apply(convert_emoji_list)

<ipython-input-102-b86244e914d2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['emoji_text']= test['content'].apply(convert_emoji_list)


In [104]:
test[4:5]

,external_author_id,author,content,region,language,publish_date,following,followers,updates,is_retweet,account_category,tweet_id,full_url,has_URL,emoji_text
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,1050,9645,246,1,RightTroll,914312219952861184,https://twitter.com/realDonaldTrump/status/914...,0,[flag: United States]


In [105]:
## Warning this takes a long time to run
merged['emoji_text']= merged['content'].apply(convert_emoji_list)

In [109]:
merged['emoji_text'][:10]

0                                []
1                                []
2                                []
3                                []
4             [flag: United States]
5                                []
6    [honeybee, honeybee, honeybee]
7                                []
8                                []
9                                []
Name: emoji_text, dtype: object

**Feature: Emoji_count**

In [116]:
#Drop null content row
merged[merged['emoji_text'].isna()]


,external_author_id,author,content,region,language,publish_date,following,followers,updates,is_retweet,account_category,tweet_id,full_url,has_URL,emoji_text
1373547,2533001646,JASPER_FLY,NaN,United States,English,2/20/2015 10:13,181,80,716,0,HashtagGamer,568715101466808320,NaN,1,None


In [121]:
merged = merged.dropna(subset=['content'])
merged[merged['emoji_text'].isna()]

,external_author_id,author,content,region,language,publish_date,following,followers,updates,is_retweet,account_category,tweet_id,full_url,has_URL,emoji_text


In [122]:
#add another feature column: # emojis used in tweet
merged['emoji_count']= merged['emoji_text'].apply(lambda x: len(x))

<ipython-input-122-cf82453dadeb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['emoji_count']= merged['emoji_text'].apply(lambda x: len(x))


In [124]:
merged['emoji_count'].describe()

count    2.610052e+06
mean     4.487114e-02
std      4.770693e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.180000e+02
Name: emoji_count, dtype: float64

In [ ]:
#writing to csv for ease of retrieving later
merged.to_csv("merged_dataset.csv")